## Library dependencies
If you are accessing this notebook locally, you need to install some `R` packages to be able to run this notebook:

See `install.R` configuration file for the list of dependencies.

## Load dependencies and seed the random number generator
Some operations in the HSF toolbox use rng. Seed the generator for repeatability of the outputs.

In [ ]:
options(warn=-1)
library(rogme)
library(tibble)
library(ggplot2)
library(R.matlab)
library(feather)
library(plyr)
library(plotly)
library(ggridges)
library(htmlwidgets)
library(cowplot)
# 123 is not coincidental, required to obtain the same Figure-4
set.seed(123)

## Set `HSF_Inputs` path (required)
Please set `inputDir` to the absolute path of the `HSF_Inputs` folder.

In [ ]:
inputDir <- "/Users/agah/Desktop/boun-parametre/demons_data_clean2/Stats_derivatives/HSF_Inputs"

### To repeat the figure without running the analysis

You can follow on from [this cell](#Create-HSF-grid-for-comparisons-of-binary-pairings-of-C%CE%B1=%CF%83-per-subject). Otherwise, please execute the cells in order.


## Run HSF analysis 

In [ ]:
# PERFORM HIERARCHICAL SHIFT FUNCTION ANALYSIS FOR ALL SUBJECTS AND PARAMETER (alpha=sigma) COMBINATIONS

outDir <- file.path(inputDir, "HSF_Objects")
dir.create(outDir)
# Perform hierarchical shift function analysis using percentile bootstrapping. 
# Model: Strain ~ conditions + segments, where conditions are dependent measures (e.g., 44_1010).  

print("Performing the HSF analyses (non-bootstrapped)...")
for (sub in list("subA","subB","subC","subD","subE")){
    for (pairs in list("44_66","44_88","44_1010","66_88","66_1010","88_1010")){
       dataName<-paste(inputDir,.Platform$file.sep,sub,"_reduced_",pairs,".file",sep="") 
       curData <- read_feather(path=dataName)
       curData$conds <- as.factor(curData$conds) # Cast type factor as required by HSF 
# We want Viridis colorscale (Purple-->Yellow) to follow the order of segments in P-->M-->D
# This must be specified as in ln19. Otherwise, alphebetical order takes the precedence! 
# Levels describe categorical entries 
# In R, c is a function that returns a 1D vector.  
       curData$segments <- factor(curData$segments,levels =c("p4","p3","p2","p1","m1","m2","d1","d2","d3","d4"))
# Single digit format is not %2d (i.e. 01,02,..).
# Therefore, alphabetical order does not follow numeric order. 
# hsf takes alphabetical order into account. 
# This conditional statement is needed for consistent subtraction direction. 
# Default setting is reversed for the pair comparisons involving 1010.
        if (pairs=="44_1010"|pairs=="66_1010"|pairs=="88_1010"){
            obj_hsf_pb <- hsf(curData, str ~ conds + segments,todo = c(2,1))
        }else{
            obj_hsf_pb <- hsf(curData, str ~ conds + segments)
        }
       saveRDS(obj_hsf_pb,paste(outDir,.Platform$file.sep,sub,"_HSF_",pairs,".rds",sep=""))
    }
}
print(paste("Outputs are saved to: ",outDir,sep=""))


You can run the following cell to detach outputs from a scroll bar.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Convert `plot_hsf_pb` objects to `ggplotly` interactive objects
* Style individual shift functions following the color organization (purple:prox, orange:med, green:dist).
* Salmon colored zero crossing 
* Group shift will be a bold white line with pronounced markers for quantiles & HDIs.

In [ ]:
# INTERACTIVE VISUALIZATION 
# Read RDS files of HSF objects for interactive visualization using ggplot2-->ggplotly 
# This cell is to be exported to a separate notebook served by the Voila interface for efficiency. 

obj2plotly <- function(curObj){

p<-plot_hsf(curObj, viridis_option="D", ind_line_alpha=0.9, ind_line_size=1, gp_line_colour = "white", gp_point_colour="white",gp_point_size=3,gp_line_size=1.5) 
names(p$data)[names(p$data) == "participants"] <- "segment"
p$labels$colour = "segment"
p$data$segment = revalue(p$data$segment, c("1"="p4", "2"="p3","3"="p2","4"="p1","5"="m1","6"="m2","7"="d1","8"="d2","9"="d3","10"="d4"))

th <- ggplot2::theme_dark() + ggplot2::theme(
  # get rid of panel grids
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  legend.title = element_blank(),  
  # Change plot and panel background
  panel.background = element_rect(fill = "rgba(45, 59, 70,1)"),
  plot.background = element_rect(fill = "black"),
  legend.background = element_rect(fill="rgba(45, 59, 70,1)",size=0.5),  
  legend.text=element_text(color="white",size=12),  
  panel.border=element_blank(),   
  axis.line=element_blank(),
  #axis.ticks = element_blank(),
  #axis.text = element_blank(),
  axis.title = element_blank()
  ) 



k <-p  + th + aes(x = dec, y = diff, colour = segment) + ggtitle("")

k$labels$colour = ''

axx <- list(
  zeroline = TRUE,
  zerolinecolor = toRGB("salmon"),
  zerolinewidth = 3 
)
    
axy <- list(
  range = c(-2,4.5)
  # The range is fixed per panel (3 MAD from both side) to infer the relative effect size   
)    
    
# Create a Purple --> Orange --> Green divergent colorscale for segment identificaiton. 
# The convention will be followed throughout the reports.     
#style(traces=1,opacity=0.8,line = list(shape = "spline",color="#762a83",width=3)) %>%    
figObj <- ggplotly(k) %>% 
            layout(height=600,width=1000,xaxis=axx,yaxis = axy) %>% # SWITCH BACK TO 400 x 400
            style(traces=1,opacity=0.4,line = list(shape = "spline",color="#762a83",width=3)) %>%
            style(traces=2,opacity=0.4,line = list(shape = "spline",color="#9970ab",width=3)) %>% 
            style(traces=3,opacity=0.4,line = list(shape = "spline",color="#c2a5cf",width=3)) %>% 
            style(traces=4,opacity=0.4,line = list(shape = "spline",color="#e7d4e8",width=3)) %>% 
            style(traces=5,opacity=0.4,line = list(shape = "spline",color="#e08214",width=3)) %>% 
            style(traces=6,opacity=0.4,line = list(shape = "spline",color="#fdb863",width=3)) %>% 
            style(traces=7,opacity=0.4,line = list(shape = "spline",color="#e6f5d0",width=3)) %>% 
            style(traces=8,opacity=0.4,line = list(shape = "spline",color="#b8e186",width=3)) %>% 
            style(traces=9,opacity=0.4,line = list(shape = "spline",color="#7fbc41",width=3)) %>% 
            style(traces=10,opacity=0.4,line = list(shape = "spline",color="#4d9221",width=3)) %>% 
            style(traces=12,opacity=1,line = list(color="white",width=5)) %>% # group line
            style(traces=13,mode = "markers",opacity=1,marker = list(color="white",size=12)) # group markers
return(figObj)    
} #END FUNCTION DEF    

## Create HSF grid for comparisons of binary pairings of C<sup>α=σ</sup> per subject

!! Please make sure that you have run [this cell](#Convert-plot_hsf_pb-objects-to-ggplotly-interactive-objects) before executing the following.

Read `ggplotly` objects saved previously, style them and arrange them into a grid layout.

In [ ]:
# Use object saved from previous cell for interactive viz
outDir <- file.path(inputDir, "HSF_Objects")
visDir <- outDir

plot_list = vector("list", 30)
iter = 1

f <- list(
  family = "Courier New, monospace",
  size = 18,
  color = "#7f7f7f"
)

# Change nested loop order for subplot 
# R subplot function is column first
for (pairs in list("44_66","44_88","44_1010","66_88","66_1010","88_1010")){
                    for (sub in list("subA","subB","subC","subD","subE")){ 
        dataName<-paste(sub,"_HSF_",pairs,".rds",sep="")
        # You can see the order by commenting in the following line
        # print(dataName)                
        curObj <- readRDS(paste(visDir,.Platform$file.sep,dataName,sep=""))
        suppressMessages(pltObj <- obj2plotly(curObj))
         if (iter==1){
        suppressMessages(plot_list[[iter]] <- pltObj
                        ) 
            }else{
        suppressMessages(plot_list[[iter]] <- pltObj %>% 
            style(showlegend=FALSE))
            }
         if (iter %% 5 ==1){
             y<-list(
              title = paste("<b>",gsub("_", "-", pairs),"</b>",sep=""),
              titlefont = f)
             plot_list[[iter]]<- plot_list[[iter]] %>%
                             layout(yaxis=y)
         }  

        iter = iter +1
    }
}

## Save interactive visualization
This cell saves `HSF_GRID.html`. You can open it in your web browser to interact with the HSF results (Figure 4).

In [ ]:
suppressMessages(HSF_GRID <- subplot(plot_list,nrows = 6,margin = 0.003,shareX = FALSE,shareY = FALSE) %>%
        layout(height=1000,width=1000) %>%
        layout(xaxis=list(title=list(text="<b>A</b>",font=f),side="top")) %>%
        layout(xaxis2=list(title=list(text="<b>B</b>",font=f),side="top"))%>%
        layout(xaxis3=list(title=list(text="<b>C</b>",font=f),side="top")) %>%       
        layout(xaxis4=list(title=list(text="<b>D</b>",font=f),side="top")) %>%
        layout(xaxis5=list(title=list(text="<b>E</b>",font=f),side="top"))  %>%
        layout(yaxis=list(title=list(text="<b>44-66</b>",font=f))) %>%
        layout(yaxis6=list(title=list(text="<b>44-88</b>",font=f)))%>%
        layout(yaxis11=list(title=list(text="<b>44-1010</b>",font=f))) %>%       
        layout(yaxis16=list(title=list(text="<b>66-88</b>",font=f))) %>%
        layout(yaxis21=list(title=list(text="<b>66-1010</b>",font=f)))%>%
        layout(yaxis26=list(title=list(text="<b>88-1010</b>",font=f)))        
                )
htmlwidgets::saveWidget(HSF_GRID, "HSF_GRID.html")

## Supplementary & explanatory figures

Including the shift function explanation figure (the following cell), the following cells gives alternative ways to look at the distribution of choice. 

In [ ]:
# Slice data for Subject - C 44 & 1010 comparison at d4
# This will use the SF implementation for dependent comparison
dataName<-paste(inputDir,.Platform$file.sep,"subC","_reduced_","44_1010",".file",sep="")
curData <- read_feather(path=dataName)
s4a4 <- curData[ curData$conds == "sigma4_alpha4", ]
s10a10 <- curData[ curData$conds == "sigma10_alpha10", ]
s4a4_d4 <- s4a4[s4a4$segments == "d4", ]
s10a10_d4 <- s10a10[s10a10$segments == "d4", ]

mycol <- rgb(255, 255, 255, max = 255, alpha = 30, names = "blue50")

customTheme <- ggplot2::theme(
  panel.background = element_rect(fill = "#2d3b46",
                                colour = "skyblue",
                                size = 0.5, linetype = "solid"),
  panel.grid.major = element_blank(), 
    #panel.grid.major = element_line(size = 0.5, linetype = 'solid', color=mycol),
  panel.grid.minor = element_blank()
  )


# Keep order the same
df_44100_d4 <- mkt2(s4a4_d4$str, s10a10_d4$str)
# Group1 corresponds to 44 and 2 to 1010
ps <- plot_scat2(data = df_44100_d4,
                 formula = obs ~ gr,
                 xlabel = "",
                 ylabel = "Mod z-scores",
                 alpha = 1,
                 shape = 21,
                 colour = "white",
                 fill = "#4d9221") #> scatterplots
ps <- ps + coord_flip()
sf <- shiftdhd(data = df_44100_d4, formula = obs ~ gr, nboot = 200)
psf <- plot_sf(sf, plot_theme =3,q_line_col="#4d9221",q_line_size = 3,symb_fill=c("yellow","yellow"),symb_col="#4d9221",diffint_col="yellow", symb_size=8)

psf <- add_sf_lab(psf, sf, 
                  y_lab_nudge = -.8, 
                  text_size = 8,
                  link_col = c("yellow", "yellow"))

#> change axis labels
psf[[1]] <- psf[[1]] +  labs(x = "Group 1 quantiles of scores (a.u.)",
                             y = "Group 1 - group 2 \nquantile differences (a.u.)")

options(repr.plot.width=20,repr.plot.height=6)
hsfp <- psf[[1]] + customTheme +  ylim(0,4.5)
hsfp$data$ci_lower = c(0,0,0,0,0,0,0,0,0)
hsfp$data$ci_upper = c(0,0,0,0,0,0,0,0,0)
ggplotly(hsfp)

p <- plot_scat2(df_44100_d4,
                xlabel = "",
                ylabel = "Scores (a.u.)",
                alpha = .5,
                shape = 21,
                colour = "white",
                fill = "#4d9221"
                ) #> scatterplots
p <- p + geom_vline(xintercept=0, color="white", size=1.5) 
p <- plot_hd_links(p, sf[[1]],
                    q_size = 2,
                    md_size = 3,
                    add_rect = TRUE,
                    q_col = "papayawhip",
                    link_col = c("yellow", "yellow"),
                    rect_alpha = 0.0,
                    rect_col = "white",
                    add_lab = FALSE
                    ) #> superimposed deciles + rectangle
p <- p + coord_flip() #> flip axes


options(repr.plot.width=15,repr.plot.height=7)
p + customTheme


In [ ]:
# Slice data for Subject - C 44 & 1010 comparison at d4
# This will use the SF implementation for dependent comparison
dataName<-paste(inputDir,.Platform$file.sep,"subC","_reduced_","44_1010",".file",sep="")
curData <- read_feather(path=dataName)
s4a4 <- curData[ curData$conds == "sigma4_alpha4", ]
s10a10 <- curData[ curData$conds == "sigma10_alpha10", ]
s4a4_p4 <- s4a4[s4a4$segments == "p4", ]
s10a10_p4 <- s10a10[s10a10$segments == "p4", ]

mycol <- rgb(255, 255, 255, max = 255, alpha = 30, names = "blue50")

customTheme <- ggplot2::theme(
  panel.background = element_rect(fill = "#2d3b46", #"#2d3b46"
                                colour = "skyblue",
                                size = 0.5, linetype = "solid"),
  panel.grid.major = element_blank(), 
    #panel.grid.major = element_line(size = 0.5, linetype = 'solid', color=mycol),
  panel.grid.minor = element_blank()
  )


# Keep order the same
df_44100_p4 <- mkt2(s4a4_p4$str, s10a10_p4$str)
# Group1 corresponds to 44 and 2 to 1010
ps <- plot_scat2(data = df_44100_p4,
                 formula = obs ~ gr,
                 xlabel = "",
                 ylabel = "Mod z-scores",
                 alpha = 1,
                 shape = 21,
                 colour = "white",
                 fill = "#762a83") #> scatterplots
ps <- ps + coord_flip()
sf <- shiftdhd(data = df_44100_p4, formula = obs ~ gr, nboot = 200)
psf <- plot_sf(sf, plot_theme =3,q_line_col="#9970ab",q_line_size = 3,symb_fill=c("yellow","yellow"),symb_col="#4d9221",diffint_col="yellow", symb_size=8)

psf <- add_sf_lab(psf, sf, 
                  y_lab_nudge = -.85, 
                  text_size = 8,
                 link_col = c("lightcoral", "cyan2"))

#> change axis labels
psf[[1]] <- psf[[1]] +  labs(x = "Group 1 quantiles of scores (a.u.)",
                             y = "Group 1 - group 2 \nquantile differences (a.u.)")

options(repr.plot.width=12,repr.plot.height=7)
hsfp <- psf[[1]] + customTheme + ylim(-4,4.5) 
hsfp$data$ci_lower = c(0,0,0,0,0,0,0,0,0)
hsfp$data$ci_upper = c(0,0,0,0,0,0,0,0,0)
hsfp

p <- plot_scat2(df_44100_p4,
                xlabel = "",
                ylabel = "Scores (a.u.)",
                alpha = .3,
                shape = 21,
                colour = "white",
                fill = "#762a83"
                ) #> scatterplots
p <- p + geom_vline(xintercept=0, color="white", size=1.5) 
p <- plot_hd_links(p, sf[[1]],
                    q_size = 2,
                    md_size = 3,
                    add_rect = TRUE,
                    q_col = "yellow",
                    link_col = c("lightcoral", "cyan2"),
                    rect_alpha = 0.0,
                    rect_col = "white",
                    add_lab = FALSE
                    ) #> superimposed deciles + rectangle
p <- p + coord_flip() #> flip axes

options(repr.plot.width=15,repr.plot.height=7)
p + customTheme


In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
ggplot(df_44100_p4, aes(x = obs, y = gr)) +
 geom_density_ridges(scale = 3) + 
  scale_y_discrete(expand = c(0, 0)) +     # will generally have to set the `expand` option
  scale_x_continuous(expand = c(0, 0)) +   # for both axes to remove unneeded padding
  coord_cartesian(clip = "on") + # to avoid clipping of the very top of the top ridgeline
  theme_ridges()

In [ ]:
source("Rallfun-v30.txt")
source("wilcox_modified.txt")
source("rgar_visualization.txt")
source("rgar_utils.txt")
plot.kde_rug_dec3 <- function(data=df){
# Plot kernel density estimates + rug plots + superimposed deciles
# for 2 groups stored in a data frame
# GAR, University of Glasgow, 2016-07-09
require(plyr)
p <- ggplot(data, aes(x=data)) + geom_density(alpha=0.7,color="magenta",fill="#762a83",trim=FALSE) +
  facet_grid(gr ~ .) +
  geom_rug(color="magenta") +
  theme(legend.position="none",
        axis.text.x = element_text(size=14),
        axis.text.y = element_text(size=14),
        axis.title.x = element_text(size=16,face="bold"),
        axis.title.y = element_text(size=16,face="bold"),
        strip.text.y = element_text(size = 20, colour = "white"),
         panel.background = element_rect(fill = "#2d3b46",
                                colour = "skyblue",
                                size = 0.5, linetype = "solid"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        strip.background = element_rect(colour="darkgrey", fill="darkgrey")) +
        ylab("Density")
p
}

aa <- mkdf2(s10a10_p4$str, s4a4_p4$str)
kde <- plot.kde_rug_dec3(aa)
kde